# Odometer

Integrates the velocity of the robot over time in order to continuously obtain a pose estimate that is relative to the pose at which the integration started.

In [ ]:
%load_ext autoreload
%autoreload 2

import time
from duckietown.components import OdometryComponent, Pose2DEstimate

## Simplest scenario: No motion

In [ ]:
# create and start odometer
odometer: OdometryComponent = OdometryComponent()
odometer.start()

v, omega = 0, 0
delta_t: float = 1

for _ in range(0, 5, delta_t):
    odometer.in_speed.put((v, omega))
    time.sleep(delta_t)
    pose: Pose2DEstimate = odometer.out_pose.get()
    print(f"< Estimated pose: {pose}")
    print(f"> Driving with speed v={v}, omega={omega} for the next {delta_t} second(s)\n")

odometer.stop()

## Simple scenario: Driving in a straight line

In [ ]:
# create and start odometer
odometer: OdometryComponent = OdometryComponent()
odometer.start()

v, omega = 1, 0
delta_t: float = 1

for _ in range(0, 5, delta_t):
    odometer.in_speed.put((v, omega))
    pose: Pose2DEstimate = odometer.out_pose.get()
    print(f"< Estimated pose: {pose}")
    print(f"> Driving with speed v={v}, omega={omega} for the next {delta_t} second(s)\n")
    time.sleep(delta_t)

# stop the motion
odometer.in_speed.put((0, 0))
pose: Pose2DEstimate = odometer.out_pose.get()
print(f"< Estimated pose: {pose}")

odometer.stop()

##### NOTE: The Odometer is purely reactive

As the odometer can only estimate the motion for speeds that applied for a finite interval of time, we notice a delay of 1 "interval" in the estimation.
This is normal, the odometer only computes the pose when a new command is given. So, if you try to query for a new pose without setting a new speed your code will block indefinitely.

## Simulate time

Sometimes we want to simulate time as well. There is a separate port on the Odometry component that takes in both velocities and time of measurement.
We will also see how the estimate is a lot more precise as we are not relying on `time.sleep()` to simulate time.

In [ ]:
# create and start odometer
odometer: OdometryComponent = OdometryComponent()
odometer.start()

v, omega = 1, 0
delta_t: float = 1
T: float = 5

for t in range(0, T, delta_t):
    odometer.in_timed_speed.put((v, omega, t))
    pose: Pose2DEstimate = odometer.out_pose.get()
    print(f"< Estimated pose: {pose}")
    print(f"> Driving with speed v={v}, omega={omega} for the next {delta_t} second(s)\n")

# stop the motion
odometer.in_timed_speed.put((0, 0, T))
pose: Pose2DEstimate = odometer.out_pose.get()
print(f"< Estimated pose: {pose}")

odometer.stop()

## Steering

Let's make a full donut or radius `2` meters in `4` seconds.

In [ ]:
import numpy as np

# create and start odometer
odometer: OdometryComponent = OdometryComponent()
odometer.start()

# define period, speeds, and radius
T: float = 4
radius: float = 2.0
delta_t: float = 1

# compute linear and angular velocities needed to achieve the circle defined above
v: float = (2 * np.pi * radius) / T
omega: float = (v / radius)

for t in range(0, T, delta_t):
    odometer.in_timed_speed.put((v, omega, t))
    pose: Pose2DEstimate = odometer.out_pose.get()
    print(f"< Estimated pose: {pose}")
    print(f"> Driving with speed v={v}, omega={omega} for the next {delta_t} second(s)\n")

# stop the motion
odometer.in_timed_speed.put((0, 0, T))
pose: Pose2DEstimate = odometer.out_pose.get()
print(f"< Estimated pose: {pose}")

odometer.stop()